In [ ]:
#!pip install ipdb
#import ipdb #Esses comandos foram para importar essa biblioteca
            #que serviu de ferramenta para estudar erros do código
            #usando debug
import time
import sys #Esse comando foi para importar uma biblioteca que pode
           #"""acessar o sistema""" e forçar certas coisas como
           #forçar a interrupção do algoritmo
import traceback
import pandas as pd #Essa biblioteca foi utilizada para ter
                    #acesso aos conjuntos de dados bidimen-
                    #sionais conhecidos como DataFrames
def tecla():
    time.sleep(1)
    for k in range(4):
        time.sleep(0.25)
        print('\n'+'...                                              ...')
    input('\n'+'Pressione Enter para sair...'+'\n')
    sys.exit()
def perguntaCancelar(): #Essa função é utilizada para perguntar se o usuário
                    #deseja cancelar o programa
    letraSaida=input('\n'+'-->Sempre confira se executou tudo corretamente.'+'\n'+
                     '-->Se deseja cancelar a execução, tecle X:'+'\n') #Variável que recebe a resposta em string
    if letraSaida == 'x' or letraSaida=='X': #Se a variável tiver como dado a string correspondente a
                                       #x maiúsculo ou minúsculo...
        print('-->Para recomeçar, feche o executável e abra-o novamente.')
        tecla()  #...O programa será forçado a parar
def main():
    print('------------------------------------------------------------------------------------------------'+'\n'+
          '--------------->SEMPRE LEIA TUDO COM ATENÇÃO PARA EVITAR POSSÍVEIS ERROS<-----------------------'+'\n'+
          '------------------------------------------------------------------------------------------------'+'\n')
    print('-->Bem vindo ao comparador de picos. Algumas informações serão necessárias para fazer o sistema funcionar.'+'\n'+
          '-->Precisa-se de uma planilha onde haverá somente uma coluna que pode está nomeada com qualquer nome mas que deverá'+'\n'+
          '-->conter os ângulos dos picos do seu padrão de difração e também deverá haver uma planilha com o padrão de difração'+'\n'+
          '-->que será usado para comparar com seus dados (Chamaremos aqui esse padrão de comparação como amostra). Nele deverá conter'+'\n'+
          '-->duas colunas, a primeira também não importa seu nome mas deve conter os dados dos ângulos da amostra e a segunda deve conter'+'\n'+
          '-->os dados da intensidade. Manter essa ordem é fundamental - Primeira coluna, os ângulos e segunda coluna, as intensidades.'+'\n'+
          'É importante também que a primeira linha de cada coluna seja um título, pois caso não haja, o programa vai entender que o primeiro ângulo'+'\n'+
          'da coluna é um título ou a primeira intensidade de uma coluna é um título.'+'\n')
    numeroDeAbas=input('\n'+'-->Insira o número inteiro de abas que a planilha de comparação (Amostra) possui:'+'\n') #Essa varIável
                            #aloca quantas abas tem na planilha de CIFs que você quer comparar com planilha dos
                            #seus dados
    
    numeroDeAbas=int(numeroDeAbas) #Aqui é para converter de string para inteiro
                                   #Por padrão input armazena dados como string
    
    abas=[] #Criar uma array para alocar os nomes que cada aba deve ter na planilha de saída
    numero=0 #É importante criar essa variável como contadora para utilizar no laço de repetição
             #while seguinte, mas devido ao sys.exit() utilizar o for não deve causar muitos problemas
             #(creio), o que isenta a existência dessa variável
    segurador=0
    while segurador == 0:
        decisao=input('\n'+'-->Você deseja inserir título por título manualmente de cada aba da planilha de destino? (S ou N)'+
                     '\n'+'-->Não recomendado caso você tenha dezenas ou centenas de abas para analisar. Caso não desje'+'\n'+
                     '-->inserir manualmente, o programa pode inserir numeração de 1 até o número n de abas automaticamente.'+
                     '\n'+'-->Fazer isso não interfere na análise, é só ver qual o número da aba na planilha de comparação e'+
                     '\n'+'-->ver a correspondente no título de aba na planilha de destino.'+'\n')
        if decisao=='s' or decisao=='S':
            while numero<numeroDeAbas: #Esse laço de repetição, adiciona os nomes a string.
                           #Utilizar laço de repetição deixa o tamanho da array responsivo
                aba=input(str(numero+1)+'-->Insira o código do CIF correspondente para o título da aba na planilha de saída:'+'\n') #Inserir o nome
                abas.append(aba) #Esse comando adiciona o nome que acabou de ser inserido na array de nomes de aba
                if (numeroDeAbas-(numero+1))!=0: #Nesse laço condicional é importante que seja (numero+1)
                #caso contrário a pergunta do letraSaida aparece de forma
                #indesejada
                    perguntaCancelar()
                numero+=1 #Então tudo recomeça até atingir o número exato de abas
            segurador=1
        elif decisao=='n' or decisao=='N':
            for i in range(numeroDeAbas):
                aba=str(i+1)
                abas.append(aba)
            segurador=1
        else:
            print('Resposta inválida.'+'\n')
            segurador=0
    perguntaCancelar() #Utilização da função
    caminhoPadrao=input('-->Agora, insira o caminho da planilha com a coluna de picos do padrão de difração para análise (Seus dados).'+'\n'+
                        '---->Exemplo:.../.../nomeDoArquivo.xlsx<----'+'\n'+
                        '-->CASO AS PLANILHAS ESTEJAM NA MESMA PASTA QUE O CÓDIGO, COLOQUE SOMENTE O NOME DO ARQUIVO.'+'\n'+
                        '---->EXEMPLO: NOME-DO-ARQUIVO.XLSX<----'+'\n'+
                        '-->O recomendado é colocar o caminho inteiro mesmo que o arquivo esteja na mesma pasta do executável'+'\n'+
                        '-->Para isso, caso esteja no WINDOWS 10, a função "Copiar caminho" está disponível na barra superior próximo'+'\n'+
                        '-->à opção "Colar".'+'\n'+
                        '-->Caso esteja no WINDOWS 11, simplesmente selecionar o arquivo e apertar o atalho "Ctrl+shift+C" é o suficiente'+'\n'+
                        '-->ou clicar com o botão esquerdo do mouse e escolher essa função.'+'\n'+
                        '---->APÓS TER CONSEGUIDO O CAMINHO, LEMBRE-SE DE RETIRAR AS ASPAS DUPLAS QUE VÊM JUNTO.<----'+'\n')
    caminhoCIFs=input('-->Também insira o caminho da planilha com a coluna de picos que deseja comparar com seu padrão de difração.'+'\n'+
                      '---->Exemplo:.../.../nomeDoArquivo.xlsx<----'+'\n'+
                      '-->CASO AS PLANILHAS ESTEJAM NA MESMA PASTA QUE O CÓDIGO, COLOQUE SOMENTE O NOME DO ARQUIVO.'+'\n'+
                      '---->EXEMPLO: NOME-DO-ARQUIVO.XLSX<----'+'\n'+
                      '-->O recomendado é colocar o caminho inteiro mesmo que o arquivo esteja na mesma pasta do executável'+'\n'+
                      '-->Para isso, caso esteja no WINDOWS 10, a função "Copiar caminho" está disponível na barra superior próximo'+'\n'+
                      '-->à opção "Colar".'+'\n'+
                      '-->Caso esteja no WINDOWS 11, simplesmente selecionar o arquivo e apertar o atalho "Ctrl+shift+C" é o suficiente'+'\n'+
                      '-->ou clicar com o botão esquerdo do mouse e escolher essa função.'+'\n'+
                      '---->APÓS TER CONSEGUIDO O CAMINHO, LEMBRE-SE DE RETIRAR AS ASPAS DUPLAS QUE VÊM JUNTO.<----'+'\n')
    caminhoPlanilhaSaida=input('-->Por fim, o caminho com o nome do arquivo para onde deve ser destinado a planilha com as comparações feitas com seu'+'\n'+
                               '-->padrão de difração. (Planilha de saída)'+'\n'+
                               '-->LEMBRE-SE que caso informe o mesmo nome que um dos arquivos presentes na pasta a qual destinou a planinha de saída, a mesma vai'+'\n'+
                               '-->substituir o arquivo que existia antes da planilha de saída'+'\n'+
                               '---->LEMBRE-SE DO .XLSX, EXEMPLO: PLANILHA-FINAL.XLSX<----'+'\n')
    #Variaveis que vão receber e adicionar os caminhos em string para utilizar mais na frente
    perguntaCancelar()
    print('-->Isso pode demorar alguns minutos...'+'\n'+
         '-->Ao fim a planilha estará na pasta que você designou'+'\n'+
         '-->bem como com as abas que você entitulou')
    #Criacao da tabela que armazenara as linhas criadas
    dataFrameResultados=pd.DataFrame(columns=['2theta-Padrão','2theta','Intensidade','Diferença','Corresponde','Picos Excedentes',
                                              'Picos Faltantes','Nota'])
    #Criação do DataFrame modelo vazio com todas as suas colunas correspondentes, tal modelo é que será utilizado em cada aba da planilha de saída
    
    array_de_DataFrames=[] #Aqui é o array de DataFrames modelo, tal arranjo é importante para haver uma distinção autônoma de cada dataFrame para cada
                           #aba construída na planilha de saída
    
    for i in range(numeroDeAbas): #A função desse laço é criar cópias do DataFrame modelo e adicionar
                                  #as mesmas ao array
        dataFrameResultadosCopia=dataFrameResultados.copy() #Aqui a cópia é feita
        array_de_DataFrames.append(dataFrameResultadosCopia) #Aqui a cópia é adicionada ao array
    #Então tudo se repete até o número da quantidade de abas da planilha de saída ser alcançado
    
    #Nesse próximo for começa a análise de fato, o primeiro for ler os dados das planilhas que você
    #passou e então chega no while que seleciona a linha da amostra que você quer comparar com o
    #seu padrão de difração e por fim o segundo while fica selecionando cada linha do padrão e com-
    #para todas elas com a linha escolhida no primeiro while
    for numeroAba in range(numeroDeAbas):
        #Criando variável que lerá os dados de pico na coluna 2theta da planilha do seu padrão designado
        tabelaPadrao = pd.read_excel(caminhoPadrao)
        #Trazendo a aba dos dados para comparacao
        tabelaAmostra = pd.read_excel(caminhoCIFs,sheet_name=numeroAba) #sheet_name informa de qual aba deve ser lido os dados
        #Criando variável para endereço da linha da amostra que vai ser comparada
        indexLinhaAmostra=0
        #Variável para o número máximo de iterações
        numeroLinhas=tabelaAmostra.iloc[:,0].count() #A função .iloc localiza linhas e colunas pelo seu índice
                                    #numérico que começa no 0, o : é para indicar que todas as linhas devem ser contadas
                                    #e o 0 é para indicar o índice da coluna (Que se espera ser a dos ângulos) a qual deve 
                                    #ser contada todas as linhas que não estão vazias, assim [linha,coluna]
                                    #e a função .count() é responsável pela tarefa de
                                    #contar essas linhas que não estão vazias
    
        #Variável para o indíce da linha que ajudará na adição de linhas para a tabela que foi criada
        contador=0
        #Variável que vai guardar o número de picos que não batem com o padrão
        picoExcedente=0
        #Essas variáveis...
        muitoBom=0
        bom=0
        medio=0
        poucoBom=0
        menosBom=0
      #...são para armazenar a quantidade de picos respectivos à cada classificação
        nota=0 #Uma variável para armazenar a nota individual de certo pico
        while indexLinhaAmostra < numeroLinhas:
            #Variável que vai guardar o ângulo do pico que será analisado
            picoDaVez=tabelaAmostra.iloc[indexLinhaAmostra,1] #<--Mudei de 0 para 1 para corresponder a um capricho meu
            #Variável que vai guardar a intensidade do pico que será analisado. Antes era um capricho, mas após mudanças de fato é bom manter
            #esse índice
            intensidadePicoAmostra=tabelaAmostra.iloc[indexLinhaAmostra,3] #<--Mudei de 1 para 3 para corresponder a um capricho meu
            #Antes era um capricho, mas após mudanças de fato é bom manter esse índice
            
            #Criando variável para saber quantas linhas há na coluna em questão para servir de quantidade final ao while
            numeroLinhasPadrao=tabelaPadrao.iloc[:,0].count()
            #Criando variável para o endereço de linha do seu padrão de difração que vai variar no laço while
            indexLinhaPadrao=0
            #Variável que vai guardar o número de rejeições a um dado pico da amostra
            numeroRejeicoes=0
            while indexLinhaPadrao < numeroLinhasPadrao:
                #Criando variável para selecionar a linha do seu padrão que será comparado com a linha da amostra da vez
                linhaPico=tabelaPadrao.iloc[indexLinhaPadrao,0]
                #Criando a variável que vai analisar o quão distante estão os ângulos
                
                diferenca=abs(linhaPico-picoDaVez) #A função abs() retorna o valor absoluto de um certo valor, logo mesmo que
                                                 #essa subtração dê um valor negativo, virá o valor absoluto dela
                if diferenca <= 0.1:
                    muitoBom+=1
                    nota=1
                    array_de_DataFrames[numeroAba].loc[contador]=[linhaPico,picoDaVez,intensidadePicoAmostra,diferenca,'Sim','-','-','-']
                elif 0.1 < diferenca <= 0.2:
                    bom+=1
                    nota=0.8
                    array_de_DataFrames[numeroAba].loc[contador]=[linhaPico,picoDaVez,intensidadePicoAmostra,diferenca,'Sim','-','-','-']
                elif 0.2 < diferenca <= 0.3:
                    medio+=1
                    nota=0.6
                    array_de_DataFrames[numeroAba].loc[contador]=[linhaPico,picoDaVez,intensidadePicoAmostra,diferenca,'Sim','-','-','-']
                elif 0.3 < diferenca <= 0.4:
                    poucoBom+=1
                    nota=0.4
                    array_de_DataFrames[numeroAba].loc[contador]=[linhaPico,picoDaVez,intensidadePicoAmostra,diferenca,'Sim','-','-','-']
                elif 0.4 < diferenca <= 0.5:
                    menosBom+=1
                    nota=0.2
                    array_de_DataFrames[numeroAba].loc[contador]=[linhaPico,picoDaVez,intensidadePicoAmostra,diferenca,'Sim','-','-','-']
                #Cada loc tem como indície de linha a variável contador que sempre vai somando 1 ao fim de cada linha padrão comparada
                #com a linha amostra, utilizar o contador como indície de linha no DataFrame é importante pois assim toda vez que um
                #dos condicionais forem satisfeitos, o índicie utilizado será um indície indisponível no DataFrame o que faz a função
                #.loc[] adicionar essa linha
                else:
                    numeroRejeicoes+=1
                #Quando nenhum dos condicionais é satisfeito, é adicionado 1 na variável acima
                if numeroRejeicoes == numeroLinhasPadrao:
                #E caso esse número de rejeições seja igual ao número de linhas do padrão, ou seja, caso
                #todos os ângulos do padrão rejeitem esse ângulo da amostra, é adicionado um na variável abaixo
                #que indica a quantidade de picos que não correspondem com o padrão
                    picoExcedente+=1
                    nota=0
                #Por fim, esse ângulo é exposto com uma linha só para ele
                    array_de_DataFrames[numeroAba].loc[contador]=['-',picoDaVez,intensidadePicoAmostra,'-','Não','-','-','-']
                contador+=1
                indexLinhaPadrao+=1
            indexLinhaAmostra += 1
        mediaColunaNotas=array_de_DataFrames[numeroAba].iloc[:,7].mean()
        contador+=1
        #Cada tabela deve ter uma linha com informações no final:
        array_de_DataFrames[numeroAba].loc[contador]=[f'Quantidade\nde Muito bom(s):\n{muitoBom}',f'Quantidade\nde Bom(s):\n{bom}',
                                                      f'Quantidade\nde Médio(s):\n{medio}',f'Quantidade\nde Pouco bom(s):\n{poucoBom}',
                                                      f'Quantidade\nde Menos bom(s):\n{menosBom}',
                                                      f'Quantidade\nde picos\nexcedentes:\n{picoExcedente}',
                                                      f'Quantidade\nde picos\nfaltantes:\n{picoFaltante}',f'Nota\nfinal\né:{mediaColunaNotas}']
      #ipdb.set_trace() #Um comando para
                        #aquela biblioteca 
                        #de debug, basicamente um breakpoint,
                        #ponto de parada para analisar o
                        #código e seu comportamento
                        #bem como os valores das variáveis naquele
                        #momento em que o breakpoint se encontra
    
    with pd.ExcelWriter(caminhoPlanilhaSaida) as writer: #Cria-se um objeto para a planilha saída
    
    #Basicamente, ao fim das análises, os DataFrames criados são alojados na array de DataFrames e
    #cada dataFrame é endereçado a sua aba, garanta que cada aba tenha o nome correto da amostra que foi
    #comparada

        for j in range(numeroDeAbas): #Aqui esse for é para garantir que diferentes dataFrames estão sendo
                                    #sendo adicionados em abas diferentes à planilha saída
            array_de_DataFrames[j].to_excel(writer,sheet_name=abas[j],index=False)
    tecla()
if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print("\n"+"----------------------------------------------------- "+
              "\n"+"\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\"+
              "\n"+"---------------------Ocorreu um erro:-----------------"+
              "\n"+"//////////////////////////////////////////////////////"+
              "\n"+" -----------------------------------------------------")
        print(e)
        traceback.print_exc()
        tecla()
    

Testes para funcionalidades futuras

In [ ]:
import pandas as pd
import numpy as np

# Criação de exemplos de DataFrames
df1 = pd.DataFrame({'A': [1, 0, 20], 'B': [4, 5, 6]})
df2 = pd.DataFrame({'A': [3, 1, 1], 'B': [6, 5, 4]})
df3 = pd.DataFrame({'A': [2, 57, 9], 'B': [5, 6, 4]})

# Lista de DataFrames
lista_dfs = [df1, df2, df3]

# Ordenar a lista de DataFrames pela média dos valores na coluna 'A'
lista_dfs_ordenada = sorted(lista_dfs, key=lambda df: df['A'].mean(),reverse=True)

# Exibir os DataFrames ordenados
for i, df in enumerate(lista_dfs_ordenada):
    print(f'DataFrame {i+1}:\n{df}\n')


In [ ]:
import pandas as pd
df1 = pd.DataFrame({'A': [1, 0, 20], 'B': [4, 5, 6]})
mediaDaColunaA=df1.iloc[:,0].mean()
print(mediaDaColunaA)